In [1]:
from san_utils import *

import moabb,sys,pickle,os, openpyxl
from moabb.datasets import BNCI2015004
from moabb.evaluations import CrossSessionEvaluation,WithinSessionEvaluation
from moabb.paradigms import FilterBankMotorImagery,FilterBankLeftRightImagery, LeftRightImagery,MotorImagery
from moabb.pipelines.utils import FilterBank
from sklearn.model_selection import train_test_split


import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_samples,silhouette_score,calinski_harabasz_score,davies_bouldin_score


In [2]:
fmin = 4;fmax = 40; sfreq=256; n_class=5
bandwidths=[[4],[2,4],[2,4,6],[2,4,6,8]]
BW=bandwidths[3]
allfilters=getfilters(fmin,fmax,bandwidth=BW) #from MS NN IV 
# a=list(allfilters)
# a.append([8.,30.])
# a.append([7.,35.])
# allfilters=np.array(a)
key_str = ''.join(map(str, BW))# + "f"
print("key:",key_str, "|| BW:",BW )

dataset = BNCI2015004()
subjects=[1,2,3,4,5,6,7,8,9]
paradigm = MotorImagery(fmin=fmin,fmax=fmax,n_classes=n_class)
X, y, meta = paradigm.get_data(dataset=dataset, subjects=[1]) #getting just to fit label encoder
value1='word_ass';value2='right_hand'
ind0 = [i for i, value in enumerate(y) if ((value == value1) or (value ==value2))]
y=y[ind0]
le,le_label_map=label_map_fn(y)
print("le_label_ma:",le_label_map)
nchan=X.shape[1]
nband_Sel=np.arange(1,21)
subjects=[1,2,3,4,5,6,7,8,9]
pkl_sdirec="BNCI2015004/tw_fb"+key_str+"/"
pkl_sdirec

Choosing from all possible events


key: 2468 || BW: [2, 4, 6, 8]
le_label_ma: {'right_hand': 0, 'word_ass': 1}


'BNCI2015004/tw_fb2468/'

In [14]:
##### Time window select ######
timewin={}
all_timewin={}
dscore_tw={}
last_sec=int(X.shape[2]/sfreq)
timewindows=get_timewin(0,last_sec) #[(0,2),(0,2.5),(0,3),(0,3.5),(0,4),(0.5,2.5),(0.5,3),(0.5,3.5),(0.5,4),(1,3),(1,3.5),(1,4),(1.5,3.5),(1.5,4),(2,4)]
paradigm = MotorImagery(fmin=fmin,fmax=fmax,n_classes=n_class)
value1='word_ass';value2='right_hand'

for subject in subjects:
    print("subject:",subject)
    X, y, meta = paradigm.get_data(dataset=dataset, subjects=[subject])
    
    select_indices = list(np.where(meta["session"] == "session_0")[0])
    X0=X[select_indices,:];y0=y[select_indices]
    select_indices = list(np.where(meta["session"] == "session_1")[0])
    X1=X[select_indices,:];y1=y[select_indices]
    ind0 = [i for i, value in enumerate(y0) if ((value == value1) or (value ==value2))]
    ind1 = [i for i, value in enumerate(y1) if ((value == value1) or (value ==value2))]
    X_train=X0[ind0];  X_test=X1[ind1]; y_train =y0[ind0] ; y_test=y1[ind1]
    y_train = le.transform(y_train);y_test = le.transform(y_test)
    n_class=2
    print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    
    timewin[subject],all_timewin[subject],dscore_tw[subject]=best_time_dscoredm(timewindows,X_train,y_train,sfreq)#dscore based selection

Choosing from all possible events


subject: 1
(80, 30, 1793) (80,) (80, 30, 1793) (80,)
TW,Sscore: [0. 2.] 0.1668649121201755
TW,Sscore: [0.5 2.5] 0.1678378551058843
TW,Sscore: [1. 3.] 0.18008387271774332
TW,Sscore: [1.5 3.5] 0.18628372151213649
TW,Sscore: [2. 4.] 0.17915228871655645
TW,Sscore: [2.5 4.5] 0.16654414058137967
TW,Sscore: [3. 5.] 0.17059821016010762
TW,Sscore: [3.5 5.5] 0.19093615368632724
TW,Sscore: [4. 6.] 0.2140172607259138
TW,Sscore: [4.5 6.5] 0.20672714247670138
TW,Sscore: [5. 7.] 0.19759904610669035
TW,Sscore: [0.  2.5] 0.17008118543083817
TW,Sscore: [0.5 3. ] 0.16784812965602455
TW,Sscore: [1.  3.5] 0.17486866450253816
TW,Sscore: [1.5 4. ] 0.18007986270397078
TW,Sscore: [2.  4.5] 0.17179737822421678
TW,Sscore: [2.5 5. ] 0.16978065372781392
TW,Sscore: [3.  5.5] 0.178700828027749
TW,Sscore: [3.5 6. ] 0.20290600337882247
TW,Sscore: [4.  6.5] 0.2055992582669893
TW,Sscore: [4.5 7. ] 0.1947770195971699
TW,Sscore: [0. 3.] 0.1702680632516229
TW,Sscore: [0.5 3.5] 0.16192439971691017
TW,Sscore: [1. 4.] 0.17015

TW,Sscore: [0.5 2.5] 0.2808320516155031
TW,Sscore: [1. 3.] 0.3811640838048273
TW,Sscore: [1.5 3.5] 0.5558658964142121
TW,Sscore: [2. 4.] 0.6510726777485513
TW,Sscore: [2.5 4.5] 0.5773070561896643
TW,Sscore: [3. 5.] 0.5306032794128168
TW,Sscore: [3.5 5.5] 0.45370492291192716
TW,Sscore: [4. 6.] 0.45145370059998047
TW,Sscore: [4.5 6.5] 0.5439007692517124
TW,Sscore: [5. 7.] 0.582704107781864
TW,Sscore: [0.  2.5] 0.27474668065991187
TW,Sscore: [0.5 3. ] 0.3465811166019272
TW,Sscore: [1.  3.5] 0.4293199163205658
TW,Sscore: [1.5 4. ] 0.5841970801302734
TW,Sscore: [2.  4.5] 0.6407736314025445
TW,Sscore: [2.5 5. ] 0.6218108146177127
TW,Sscore: [3.  5.5] 0.5470792537474739
TW,Sscore: [3.5 6. ] 0.5017795351955388
TW,Sscore: [4.  6.5] 0.5262182660621542
TW,Sscore: [4.5 7. ] 0.6213644782774341
TW,Sscore: [0. 3.] 0.34054571563786906
TW,Sscore: [0.5 3.5] 0.3902023022764932
TW,Sscore: [1. 4.] 0.45594327689331515
TW,Sscore: [1.5 4.5] 0.579970745297547
TW,Sscore: [2. 5.] 0.6772506177063798
TW,Sscore: [2

TW,Sscore: [2. 4.] 0.1976340398971945
TW,Sscore: [2.5 4.5] 0.20297179467059753
TW,Sscore: [3. 5.] 0.19943188247235788
TW,Sscore: [3.5 5.5] 0.18805267029349487
TW,Sscore: [4. 6.] 0.18870836774137803
TW,Sscore: [4.5 6.5] 0.192287449688538
TW,Sscore: [5. 7.] 0.1857310705237738
TW,Sscore: [0.  2.5] 0.20590568773797946
TW,Sscore: [0.5 3. ] 0.2233250815499375
TW,Sscore: [1.  3.5] 0.23455600055849485
TW,Sscore: [1.5 4. ] 0.20321937654973046
TW,Sscore: [2.  4.5] 0.2068410037672058
TW,Sscore: [2.5 5. ] 0.2053354210053956
TW,Sscore: [3.  5.5] 0.1909487996817776
TW,Sscore: [3.5 6. ] 0.18661261099342566
TW,Sscore: [4.  6.5] 0.18780202954238878
TW,Sscore: [4.5 7. ] 0.18580569159135388
TW,Sscore: [0. 3.] 0.2076925340214579
TW,Sscore: [0.5 3.5] 0.21554961993086888
TW,Sscore: [1. 4.] 0.21969899572788867
TW,Sscore: [1.5 4.5] 0.21023219351016026
TW,Sscore: [2. 5.] 0.20807817297611192
TW,Sscore: [2.5 5.5] 0.19843788184612215
TW,Sscore: [3. 6.] 0.18736969801006526
TW,Sscore: [3.5 6.5] 0.18500795098743397


In [21]:
###save 
with open(pkl_sdirec+'timewin.pkl', 'wb') as file:
    pickle.dump(timewin, file)   
with open(pkl_sdirec+'all_timewin.pkl', 'wb') as file:
    pickle.dump(all_timewin, file)
with open(pkl_sdirec+'dscore_tw.pkl', 'wb') as file:
    pickle.dump(dscore_tw, file)
pkl_sdirec

'BNCI2015004/tw_fb2468/'

In [3]:
### load
with open(pkl_sdirec+'timewin.pkl', 'rb') as file:
    timewin=pickle.load(file)
with open(pkl_sdirec+'all_timewin.pkl', 'rb') as file:
    all_timewin=pickle.load( file)   
with open(pkl_sdirec+'dscore_tw.pkl', 'rb') as file:
    dscore_tw=pickle.load( file)

In [6]:
#### freq band select ######
def savepkl(pkl_sdirec,subject,sdict):
    sfilename=pkl_sdirec+str(subject)+'_scores.pkl'
    with open(sfilename, 'wb') as file:
        pickle.dump(sdict, file)
    print("saved pkl file!")
    return

sub_filters={}
all_filters={}
dscore_fb={}
n_class=5
paradigm = FilterBankMotorImagery(filters=allfilters,n_classes=n_class)
for subject in subjects:
    print("subject:",subject)
    X, y, meta = paradigm.get_data(dataset=dataset, subjects=[subject])
    s1=int(sfreq*timewin[subject][0]);     s2=int(sfreq*timewin[subject][1]) ; X=X[:,:,s1:s2]
    
    select_indices = list(np.where(meta["session"] == "session_0")[0])
    X0=X[select_indices,:];y0=y[select_indices]
    select_indices = list(np.where(meta["session"] == "session_1")[0])
    X1=X[select_indices,:];y1=y[select_indices]
    ind0 = [i for i, value in enumerate(y0) if ((value == value1) or (value ==value2))]
    ind1 = [i for i, value in enumerate(y1) if ((value == value1) or (value ==value2))]
    X0=X0[ind0];  X1=X1[ind1]; y0 =y0[ind0] ; y1=y1[ind1]
    y0 = le.transform(y0);y1 = le.transform(y1)
    n_class=2
    
    filters={}
    for nb in nband_Sel:
        filters[nb],all_filters[subject],dscore_fb[subject]=bestfilters_dscoredm(allfilters,X0,y0,nb)#dscore based selection
    sub_filters[subject]=filters
    sdict={"all_filters":all_filters,"dscore_fb":dscore_fb, "sub_filters":sub_filters,
           "le_label_map":le_label_map,"X0":X0, "y0":y0, "X1":X1, "y1":y1}
    savepkl(pkl_sdirec,subject,sdict)

Choosing from all possible events


subject: 1
saved pkl file!
subject: 2
saved pkl file!
subject: 3
saved pkl file!
subject: 4
saved pkl file!
subject: 5
saved pkl file!
subject: 6
saved pkl file!
subject: 7
saved pkl file!
subject: 8
saved pkl file!
subject: 9
saved pkl file!


In [7]:

########### save filters, dscore in .pkl file and .csv file #######
with open(pkl_sdirec+'le_label_map.pkl', 'wb') as file:
    pickle.dump(le_label_map, file)

with open(pkl_sdirec+'all_filters.pkl', 'wb') as file:
    pickle.dump(all_filters, file)
with open(pkl_sdirec+'dscore_fb.pkl', 'wb') as file:
    pickle.dump(dscore_fb, file)
with open(pkl_sdirec+'sub_filters.pkl', 'wb') as file:
    pickle.dump(sub_filters, file)      
    

In [8]:
############### load .pkl file #############3
with open(pkl_sdirec+'le_label_map.pkl', 'rb') as file:
    le_label_map=pickle.load( file)
    
with open(pkl_sdirec+'all_filters.pkl', 'rb') as file:
    all_filters=pickle.load(file)
with open(pkl_sdirec+'dscore_fb.pkl', 'rb') as file:
    dscore_fb=pickle.load( file)   
with open(pkl_sdirec+'sub_filters.pkl', 'rb') as file:
    sub_filters=pickle.load( file)


In [11]:
path = pkl_sdirec+'alltwfilters_dscore'+key_str+'.xlsx'

#############1 save excel all ######## https://xlsxwriter.readthedocs.io/example_pandas_multiple.html
writer = pd.ExcelWriter(path, engine='xlsxwriter')

df_le=pd.DataFrame([le_label_map])
df_le.to_excel(writer, sheet_name="le_map",index=False)

df_fb=pd.DataFrame.from_dict(dscore_fb)
df_fb.insert(loc = 0, column = 'sub_band', value = all_filters[1])
df_fb.to_excel(writer, sheet_name="subband_dscoredm",index=False)

df_tw=pd.DataFrame.from_dict(dscore_tw)
df_tw.insert(loc = 0, column = 'time_win', value = all_timewin[1])
df_tw.to_excel(writer, sheet_name="timewin_dscoredm",index=False)

writer.save()
writer.close()





C:\Users\jsmms\anaconda3\envs\RPA1\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [12]:
#############2 save excel best ######## https://xlsxwriter.readthedocs.io/example_pandas_multiple.html

path_best=pkl_sdirec+'Best_twfilters'+key_str+'.xlsx'
writer = pd.ExcelWriter(path_best, engine='xlsxwriter')

df_tw=pd.DataFrame()
df_tw["sub"]=timewin.keys()
df_tw["besttw"]=timewin.values()
df_tw.to_excel(writer, sheet_name="bestTW",index=False)

df_bfb = pd.DataFrame()
for sub in subjects:
    df_bfb["num_band"]=sub_filters[sub].keys()
    df_bfb[sub]=sub_filters[sub].values()
df_bfb.to_excel(writer, sheet_name="bestfb",index=False)

writer.save()
writer.close()



C:\Users\jsmms\anaconda3\envs\RPA1\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
